In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [5]:
df=pd.read_csv('../../Datasets/Twitter/final_lpu_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,#Students of Lovely Professional University ca...,The Hindu Education Plus,1138010122645409792,2019-06-10 09:08:51,"['Students', 'oncampusmall', 'industry']",TheHinduEduPlus,0,3,0.0,0.7717,Positive,5
1,1,"""The eye should learn to listen before it look...",A M I T | S H A R M A,1137740907560747008,2019-06-09 15:19:05,"['LPU', 'lpulife', 'lpu', 'university', 'beaut...",sharmabikku,0,1,0.0,0.7579,Positive,5
2,2,"Want to study B.A., LL.B.(Hons.) at Lovely Pro...",Study-India,1137683337324191744,2019-06-09 11:30:19,[],study_at_india,0,0,0.0,0.8360,Positive,5
3,3,Why study MCA (Hons.) at lovely professional u...,Study-India,1137619115873525760,2019-06-09 07:15:07,[],study_at_india,0,0,0.0,0.8225,Positive,5
4,4,sir iam sai charan from lovely professional un...,sai chara,1137247154160857088,2019-06-08 06:37:05,[],saichara7,0,0,0.0,0.8668,Positive,5
5,5,Mmm I’m not uneducated Lovely.I have Universit...,loulou#FBPE#BlackLivesMatter,1137086891373862912,2019-06-07 20:00:15,[],screene_louise,0,1,0.0,0.9597,Positive,5
6,6,3 out of 4 reviews overturned by DRS. This hap...,Rahul Singh,1136672546311401472,2019-06-06 16:33:48,['AUSvWIpic'],proudySingh,0,2,0.0,0.5859,Positive,4
7,7,How does it feel to be a student at Lovely Pro...,NdroPc,1136174928892444672,2019-06-05 07:36:26,[],NdroPc,0,0,0.0,0.6322,Positive,5
8,8,Answer to How does it feel to be a student at ...,RS.Pawar,1136174839608168448,2019-06-05 07:36:05,[],rahul_spawar,0,0,0.0,0.6322,Positive,5
9,9,Happy Eid Mubarak\n#eid @ Lovely Professional ...,HarishYemireddy,1136027341841260544,2019-06-04 21:49:59,['eid'],yharish98,0,0,0.0,0.8176,Positive,5


In [6]:
df=df.drop(['Unnamed: 0'],axis=1)

In [7]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [8]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)

In [9]:
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,#Students of Lovely Professional University ca...,The Hindu Education Plus,1138010122645409792,2019-06-10 09:08:51,"['Students', 'oncampusmall', 'industry']",TheHinduEduPlus,0.0,3.0,0.0,0.7717,Positive,5,0.000000,0.001155
1,"""The eye should learn to listen before it look...",A M I T | S H A R M A,1137740907560747008,2019-06-09 15:19:05,"['LPU', 'lpulife', 'lpu', 'university', 'beaut...",sharmabikku,0.0,1.0,0.0,0.7579,Positive,5,0.000000,0.000385
2,"Want to study B.A., LL.B.(Hons.) at Lovely Pro...",Study-India,1137683337324191744,2019-06-09 11:30:19,[],study_at_india,0.0,0.0,0.0,0.8360,Positive,5,0.000000,0.000000
3,Why study MCA (Hons.) at lovely professional u...,Study-India,1137619115873525760,2019-06-09 07:15:07,[],study_at_india,0.0,0.0,0.0,0.8225,Positive,5,0.000000,0.000000
4,sir iam sai charan from lovely professional un...,sai chara,1137247154160857088,2019-06-08 06:37:05,[],saichara7,0.0,0.0,0.0,0.8668,Positive,5,0.000000,0.000000
5,Mmm I’m not uneducated Lovely.I have Universit...,loulou#FBPE#BlackLivesMatter,1137086891373862912,2019-06-07 20:00:15,[],screene_louise,0.0,1.0,0.0,0.9597,Positive,5,0.000000,0.000385
6,3 out of 4 reviews overturned by DRS. This hap...,Rahul Singh,1136672546311401472,2019-06-06 16:33:48,['AUSvWIpic'],proudySingh,0.0,2.0,0.0,0.5859,Positive,4,0.000000,0.000770
7,How does it feel to be a student at Lovely Pro...,NdroPc,1136174928892444672,2019-06-05 07:36:26,[],NdroPc,0.0,0.0,0.0,0.6322,Positive,5,0.000000,0.000000
8,Answer to How does it feel to be a student at ...,RS.Pawar,1136174839608168448,2019-06-05 07:36:05,[],rahul_spawar,0.0,0.0,0.0,0.6322,Positive,5,0.000000,0.000000
9,Happy Eid Mubarak\n#eid @ Lovely Professional ...,HarishYemireddy,1136027341841260544,2019-06-04 21:49:59,['eid'],yharish98,0.0,0.0,0.0,0.8176,Positive,5,0.000000,0.000000


In [10]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [11]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [12]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [13]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [14]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,#students of lovely professional university ca...,The Hindu Education Plus,1138010122645409792,2019-06-10 09:08:51,"['Students', 'oncampusmall', 'industry']",TheHinduEduPlus,0.0,3.0,0.0,0.7717,Positive,5,0.000000,0.001155,0.046207,#Students of Lovely Professional University ca...,"[students, lovely, professional, university, s..."
1,"""the eye should learn to listen before it look...",A M I T | S H A R M A,1137740907560747008,2019-06-09 15:19:05,"['LPU', 'lpulife', 'lpu', 'university', 'beaut...",sharmabikku,0.0,1.0,0.0,0.7579,Positive,5,0.000000,0.000385,0.015402,"""The eye should learn to listen before it look...","[eye, learn, listen, looks, sharmabikku, speci..."
2,"want to study b.a., ll.b.(hons.) at lovely pro...",Study-India,1137683337324191744,2019-06-09 11:30:19,[],study_at_india,0.0,0.0,0.0,0.8360,Positive,5,0.000000,0.000000,0.000000,Want to study B.A. LL.B.(Hons. at Lovely Profe...,"[want, study, b, b, hons, lovely, professional..."
3,why study mca (hons.) at lovely professional u...,Study-India,1137619115873525760,2019-06-09 07:15:07,[],study_at_india,0.0,0.0,0.0,0.8225,Positive,5,0.000000,0.000000,0.000000,Why study MCA (Hons. at lovely professional un...,"[study, mca, hons, lovely, professional, unive..."
4,sir iam sai charan from lovely professional un...,sai chara,1137247154160857088,2019-06-08 06:37:05,[],saichara7,0.0,0.0,0.0,0.8668,Positive,5,0.000000,0.000000,0.000000,sir iam sai charan from lovely professional un...,"[sir, iam, sai, charan, lovely, professional, ..."
5,mmm i’m not uneducated lovely.i have universit...,loulou#FBPE#BlackLivesMatter,1137086891373862912,2019-06-07 20:00:15,[],screene_louise,0.0,1.0,0.0,0.9597,Positive,5,0.000000,0.000385,0.015402,Mmm I’m not uneducated Lovely.I have Universit...,"[mmm, uneducated, lovely, university, degree, ..."
6,3 out of 4 reviews overturned by drs. this hap...,Rahul Singh,1136672546311401472,2019-06-06 16:33:48,['AUSvWIpic'],proudySingh,0.0,2.0,0.0,0.5859,Positive,4,0.000000,0.000770,0.030805,3 out of 4 reviews overturned by DRS This happ...,"[3, 4, reviews, overturned, drs, happens, pass..."
7,how does it feel to be a student at lovely pro...,NdroPc,1136174928892444672,2019-06-05 07:36:26,[],NdroPc,0.0,0.0,0.0,0.6322,Positive,5,0.000000,0.000000,0.000000,How does it feel to be a student at Lovely Pro...,"[feel, student, lovely, professional, universi..."
8,answer to how does it feel to be a student at ...,RS.Pawar,1136174839608168448,2019-06-05 07:36:05,[],rahul_spawar,0.0,0.0,0.0,0.6322,Positive,5,0.000000,0.000000,0.000000,Answer to How does it feel to be a student at ...,"[answer, feel, student, lovely, professional, ..."
9,happy eid mubarak\n#eid @ lovely professional ...,HarishYemireddy,1136027341841260544,2019-06-04 21:49:59,['eid'],yharish98,0.0,0.0,0.0,0.8176,Positive,5,0.000000,0.000000,0.000000,Happy Eid Mubarak #eid Lovely Professional Uni...,"[happy, eid, mubarak, eid, lovely, professiona..."


In [15]:
nltk.download('averaged_perceptron_tagger')
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Manam/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,#students of lovely professional university ca...,The Hindu Education Plus,1138010122645409792,2019-06-10 09:08:51,"['Students', 'oncampusmall', 'industry']",TheHinduEduPlus,0.0,3.0,0.0,0.7717,Positive,5,0.000000,0.001155,0.046207,#Students of Lovely Professional University ca...,"[students, lovely, professional, university, s...","[(students, NNS), (lovely, JJ), (professional,..."
1,"""the eye should learn to listen before it look...",A M I T | S H A R M A,1137740907560747008,2019-06-09 15:19:05,"['LPU', 'lpulife', 'lpu', 'university', 'beaut...",sharmabikku,0.0,1.0,0.0,0.7579,Positive,5,0.000000,0.000385,0.015402,"""The eye should learn to listen before it look...","[eye, learn, listen, looks, sharmabikku, speci...","[(eye, NN), (learn, NN), (listen, JJ), (looks,..."
2,"want to study b.a., ll.b.(hons.) at lovely pro...",Study-India,1137683337324191744,2019-06-09 11:30:19,[],study_at_india,0.0,0.0,0.0,0.8360,Positive,5,0.000000,0.000000,0.000000,Want to study B.A. LL.B.(Hons. at Lovely Profe...,"[want, study, b, b, hons, lovely, professional...","[(want, NN), (study, NN), (b, NN), (b, NN), (h..."
3,why study mca (hons.) at lovely professional u...,Study-India,1137619115873525760,2019-06-09 07:15:07,[],study_at_india,0.0,0.0,0.0,0.8225,Positive,5,0.000000,0.000000,0.000000,Why study MCA (Hons. at lovely professional un...,"[study, mca, hons, lovely, professional, unive...","[(study, NN), (mca, NN), (hons, NNS), (lovely,..."
4,sir iam sai charan from lovely professional un...,sai chara,1137247154160857088,2019-06-08 06:37:05,[],saichara7,0.0,0.0,0.0,0.8668,Positive,5,0.000000,0.000000,0.000000,sir iam sai charan from lovely professional un...,"[sir, iam, sai, charan, lovely, professional, ...","[(sir, NN), (iam, NN), (sai, NN), (charan, NN)..."
5,mmm i’m not uneducated lovely.i have universit...,loulou#FBPE#BlackLivesMatter,1137086891373862912,2019-06-07 20:00:15,[],screene_louise,0.0,1.0,0.0,0.9597,Positive,5,0.000000,0.000385,0.015402,Mmm I’m not uneducated Lovely.I have Universit...,"[mmm, uneducated, lovely, university, degree, ...","[(mmm, NN), (uneducated, VBD), (lovely, RB), (..."
6,3 out of 4 reviews overturned by drs. this hap...,Rahul Singh,1136672546311401472,2019-06-06 16:33:48,['AUSvWIpic'],proudySingh,0.0,2.0,0.0,0.5859,Positive,4,0.000000,0.000770,0.030805,3 out of 4 reviews overturned by DRS This happ...,"[3, 4, reviews, overturned, drs, happens, pass...","[(3, CD), (4, CD), (reviews, NNS), (overturned..."
7,how does it feel to be a student at lovely pro...,NdroPc,1136174928892444672,2019-06-05 07:36:26,[],NdroPc,0.0,0.0,0.0,0.6322,Positive,5,0.000000,0.000000,0.000000,How does it feel to be a student at Lovely Pro...,"[feel, student, lovely, professional, universi...","[(feel, VB), (student, NN), (lovely, RB), (pro..."
8,answer to how does it feel to be a student at ...,RS.Pawar,1136174839608168448,2019-06-05 07:36:05,[],rahul_spawar,0.0,0.0,0.0,0.6322,Positive,5,0.000000,0.000000,0.000000,Answer to How does it feel to be a student at ...,"[answer, feel, student, lovely, professional, ...","[(answer, NN), (feel, VB), (student, NN), (lov..."
9,happy eid mubarak\n#eid @ lovely professional ...,HarishYemireddy,1136027341841260544,2019-06-04 21:49:59,['eid'],yharish98,0.0,0.0,0.0,0.8176,Positive,5,0.000000,0.000000,0.000000,Happy Eid Mubarak #eid Lovely Professional Uni...,"[happy, eid, mubarak, eid, lovely, professiona...","[(happy, JJ), (eid, NN), (mubarak, NN), (eid, ..."


In [16]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,#students of lovely professional university ca...,The Hindu Education Plus,1138010122645409792,2019-06-10 09:08:51,"['Students', 'oncampusmall', 'industry']",TheHinduEduPlus,0.0,3.0,0.0,0.7717,Positive,5,0.000000,0.001155,0.046207,#Students of Lovely Professional University ca...,"[students, lovely, professional, university, s...","[(students, NNS), (lovely, JJ), (professional,...","[students, university, transition, sector, tha..."
1,"""the eye should learn to listen before it look...",A M I T | S H A R M A,1137740907560747008,2019-06-09 15:19:05,"['LPU', 'lpulife', 'lpu', 'university', 'beaut...",sharmabikku,0.0,1.0,0.0,0.7579,Positive,5,0.000000,0.000385,0.015402,"""The eye should learn to listen before it look...","[eye, learn, listen, looks, sharmabikku, speci...","[(eye, NN), (learn, NN), (listen, JJ), (looks,...","[eye, learn, capture, memory, lpulife, lpu, un..."
2,"want to study b.a., ll.b.(hons.) at lovely pro...",Study-India,1137683337324191744,2019-06-09 11:30:19,[],study_at_india,0.0,0.0,0.0,0.8360,Positive,5,0.000000,0.000000,0.000000,Want to study B.A. LL.B.(Hons. at Lovely Profe...,"[want, study, b, b, hons, lovely, professional...","[(want, NN), (study, NN), (b, NN), (b, NN), (h...","[want, study, b, b, hons, university, requirem..."
3,why study mca (hons.) at lovely professional u...,Study-India,1137619115873525760,2019-06-09 07:15:07,[],study_at_india,0.0,0.0,0.0,0.8225,Positive,5,0.000000,0.000000,0.000000,Why study MCA (Hons. at lovely professional un...,"[study, mca, hons, lovely, professional, unive...","[(study, NN), (mca, NN), (hons, NNS), (lovely,...","[study, mca, hons, university, eligibility, re..."
4,sir iam sai charan from lovely professional un...,sai chara,1137247154160857088,2019-06-08 06:37:05,[],saichara7,0.0,0.0,0.0,0.8668,Positive,5,0.000000,0.000000,0.000000,sir iam sai charan from lovely professional un...,"[sir, iam, sai, charan, lovely, professional, ...","[(sir, NN), (iam, NN), (sai, NN), (charan, NN)...","[sir, iam, sai, charan, university, idea, star..."
5,mmm i’m not uneducated lovely.i have universit...,loulou#FBPE#BlackLivesMatter,1137086891373862912,2019-06-07 20:00:15,[],screene_louise,0.0,1.0,0.0,0.9597,Positive,5,0.000000,0.000385,0.015402,Mmm I’m not uneducated Lovely.I have Universit...,"[mmm, uneducated, lovely, university, degree, ...","[(mmm, NN), (uneducated, VBD), (lovely, RB), (...","[mmm, qualification, sure, farage, fighting, f..."
6,3 out of 4 reviews overturned by drs. this hap...,Rahul Singh,1136672546311401472,2019-06-06 16:33:48,['AUSvWIpic'],proudySingh,0.0,2.0,0.0,0.5859,Positive,4,0.000000,0.000770,0.030805,3 out of 4 reviews overturned by DRS This happ...,"[3, 4, reviews, overturned, drs, happens, pass...","[(3, CD), (4, CD), (reviews, NNS), (overturned...","[reviews, happens, course, university, rip, au..."
7,how does it feel to be a student at lovely pro...,NdroPc,1136174928892444672,2019-06-05 07:36:26,[],NdroPc,0.0,0.0,0.0,0.6322,Positive,5,0.000000,0.000000,0.000000,How does it feel to be a student at Lovely Pro...,"[feel, student, lovely, professional, universi...","[(feel, VB), (student, NN), (lovely, RB), (pro...","[student, university, lpu]"
8,answer to how does it feel to be a student at ...,RS.Pawar,1136174839608168448,2019-06-05 07:36:05,[],rahul_spawar,0.0,0.0,0.0,0.6322,Positive,5,0.000000,0.000000,0.000000,Answer to How does it feel to be a student at ...,"[answer, feel, student, lovely, professional, ...","[(answer, NN), (feel, VB), (student, NN), (lov...","[answer, student, university, lpu, rahul, pawar]"
9,happy eid mubarak\n#eid @ lovely professional ...,HarishYemireddy,1136027341841260544,2019-06-04 21:49:59,['eid'],yharish98,0.0,0.0,0.0,0.8176,Positive,5,0.000000,0.000000,0.000000,Happy Eid Mubarak #eid Lovely Professional Uni...,"[happy, eid, mubarak, eid, lo

In [17]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,students
1,university
2,transition
3,sector
4,thanks
5,industry
6,article
7,eye
8,learn
9,capture


In [18]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,students,students,1059
1,university,university,4035
2,transition,transition,2
3,sector,sector,5
4,thanks,thanks,54
5,industry,industry,8
6,article,article,2
7,eye,eye,5
8,learn,learn,3
9,capture,capture,3


In [19]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
1,university,university,4035
0,students,students,1059
12,lpu,lpu,960
61,punjab,punjab,726
45,com,com,598
44,twitter,twitter,595
105,please,please,519
24,sir,sir,513
46,student,student,485
104,jalandhar,jalandhar,352


In [20]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [21]:
#list of irreleveant tweet topics
l=['happybirthdaypawankalyanpic','janasenani','hemantsorenjmm','indiabasketballpic']

In [22]:
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))

In [23]:
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
401,3bl season 2 opens in the presence of the hono...,3X3BL,1157268872094838784,2019-08-02 12:36:14,"['LetsGetGold', '3x3', '3x3BL', '3x3India', 'S...",3X3BL,0.0,0.0,0.0,0.8221,Positive,5,0.000000,0.000000,0.000000,3BL Season 2 opens in the presence of the Hono...,"[3bl, season, 2, opens, presence, honorable, s...","[(3bl, CD), (season, NN), (2, CD), (opens, NNS...","[season, opens, presence, shri, rana, gurmeet,...",1
420,3bl season 2 opens in the presence of the hono...,3X3BL,1157268872094838784,2019-08-02 12:36:14,"['LetsGetGold', '3x3', '3x3BL', '3x3India', 'S...",3X3BL,0.0,0.0,0.0,0.8221,Positive,5,0.000000,0.000000,0.000000,3BL Season 2 opens in the presence of the Hono...,"[3bl, season, 2, opens, presence, honorable, s...","[(3bl, CD), (season, NN), (2, CD), (opens, NNS...","[season, opens, presence, shri, rana, gurmeet,...",1
795,this arena looks all thanks to the support we...,3X3BL,1162372322038181888,2019-08-16 14:35:32,"['LetsGetGold', '3x3', '3x3BL', 'Season2', '3x...",3X3BL,0.0,1.0,0.0,0.8555,Positive,5,0.000000,0.000385,0.015402,This arena looks all thanks to the support we’...,"[arena, looks, thanks, support, received, venu...","[(arena, JJ), (looks, VBZ), (thanks, NNS), (su...","[thanks, support, partner, university, lpu, lp...",1
799,we're back again at lovely professional univer...,3X3BL,1162222657435131904,2019-08-16 04:40:49,"['LIVE', 'LetsGetGold', '3x3', '3x3BL', '3x3BL...",3X3BL,3.0,6.0,0.0,0.2714,Positive,4,0.001373,0.002310,0.174794,We're back again at Lovely Professional Univer...,"[back, lovely, professional, university, lpuun...","[(back, RB), (lovely, JJ), (professional, JJ),...","[university, lpuuniversity, looks, battle, rev...",1
962,the wait is over! \ncatch some adrenaline pump...,3X3BL,1172172479256616960,2019-09-12 15:37:51,"['LIVE', 'LetsGetGold', '3x3', '3x3BL', 'Seaso...",3X3BL,0.0,0.0,0.0,0.6239,Positive,5,0.000000,0.000000,0.000000,The wait is over Catch some adrenaline pumping...,"[wait, catch, adrenaline, pumping, action, rou...","[(wait, NN), (catch, NN), (adrenaline, NN), (p...","[wait, catch, adrenaline, action, university, ...",1
982,the wait is over! \ncatch some adrenaline pump...,3X3BL,1172172479256616960,2019-09-12 15:37:51,"['LIVE', 'LetsGetGold', '3x3', '3x3BL', 'Seaso...",3X3BL,0.0,0.0,0.0,0.6239,Positive,5,0.000000,0.000000,0.000000,The wait is over Catch some adrenaline pumping...,"[wait, catch, adrenaline, pumping, action, rou...","[(wait, NN), (catch, NN), (adrenaline, NN), (p...","[wait, catch, adrenaline, action, university, ...",1
1055,lovely professional university mega guys suppl...,FAN OF KALYAN 🦁,1168438956838440960,2019-09-02 08:22:10,['HappyBirthdayPawanKalyanpic'],Saichandupspk6,1.0,0.0,0.0,0.5859,Positive,4,0.000458,0.000000,0.027460,Lovely professional university mega guys suppl...,"[lovely, professional, university, mega, guys,...","[(lovely, RB), (professional, JJ), (university...","[university, guys, food, orphanage, punjab, me...",1
1057,janasenani birthday celebrations by lovely pro...,Ankipalli Sivaji,1168362629955543040,2019-09-02 03:18:52,['HappyBirthdayPawanKalyanpic'],AnkipalliSivaj1,11.0,0.0,0.0,0.5859,Positive,4,0.005034,0.000000,0.302059,Janasenani Birthday Celebrations by Lovely Pro...,"[janasenani, birthday, celebrations, lovely, p...","[(janasenani, NN), (birthday, NN), (celebratio...","[janasenani, birthday, celebrations, universit...",1
1059,janasenani birthday celebrations by lovely pro...,TREND SETTER SIDDHU,1168350784745947136,2019-09-02 02:31:48,['HappyBirthdayPawanKalyanpic'],siddhuvenkat721,1.0,0.0,0.0,0.5859,Positive,4,0.000458,0.000000,0.027460,Janasenani Birthday Celebrations by Lovely Pro...,"[janasenani, birthday, celebrations, lovely, p...","[(janasenani, NN), (birthday, NN), (celebratio...","[janasenani, birthday, celebrations, universit...

In [24]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [28]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [40]:
df['new_score']=df['score']

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [41]:
df_eng1=df[df.engagement==1]

In [42]:
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [43]:
df[df.engagement==1]=df_eng1

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/Manam/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [44]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [45]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [46]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

/Users/Manam/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,#students of lovely professional university ca...,The Hindu Education Plus,1138010122645409792,2019-06-10 09:08:51,"['Students', 'oncampusmall', 'industry']",TheHinduEduPlus,0.0,3.0,0.0,0.7717,...,0.000000,0.001155,0.046207,#Students of Lovely Professional University ca...,"[students, lovely, professional, university, s...","[(students, NNS), (lovely, JJ), (professional,...","[students, university, transition, sector, tha...",0,1,5
1,"""the eye should learn to listen before it look...",A M I T | S H A R M A,1137740907560747008,2019-06-09 15:19:05,"['LPU', 'lpulife', 'lpu', 'university', 'beaut...",sharmabikku,0.0,1.0,0.0,0.7579,...,0.000000,0.000385,0.015402,"""The eye should learn to listen before it look...","[eye, learn, listen, looks, sharmabikku, speci...","[(eye, NN), (learn, NN), (listen, JJ), (looks,...","[eye, learn, capture, memory, lpulife, lpu, un...",0,0,5
2,"want to study b.a., ll.b.(hons.) at lovely pro...",Study-India,1137683337324191744,2019-06-09 11:30:19,[],study_at_india,0.0,0.0,0.0,0.8360,...,0.000000,0.000000,0.000000,Want to study B.A. LL.B.(Hons. at Lovely Profe...,"[want, study, b, b, hons, lovely, professional...","[(want, NN), (study, NN), (b, NN), (b, NN), (h...","[want, study, b, b, hons, university, requirem...",0,0,5
3,why study mca (hons.) at lovely professional u...,Study-India,1137619115873525760,2019-06-09 07:15:07,[],study_at_india,0.0,0.0,0.0,0.8225,...,0.000000,0.000000,0.000000,Why study MCA (Hons. at lovely professional un...,"[study, mca, hons, lovely, professional, unive...","[(study, NN), (mca, NN), (hons, NNS), (lovely,...","[study, mca, hons, university, eligibility, re...",0,0,5
4,sir iam sai charan from lovely professional un...,sai chara,1137247154160857088,2019-06-08 06:37:05,[],saichara7,0.0,0.0,0.0,0.8668,...,0.000000,0.000000,0.000000,sir iam sai charan from lovely professional un...,"[sir, iam, sai, charan, lovely, professional, ...","[(sir, NN), (iam, NN), (sai, NN), (charan, NN)...","[sir, iam, sai, charan, university, idea, star...",0,0,5
5,mmm i’m not uneducated lovely.i have universit...,loulou#FBPE#BlackLivesMatter,1137086891373862912,2019-06-07 20:00:15,[],screene_louise,0.0,1.0,0.0,0.9597,...,0.000000,0.000385,0.015402,Mmm I’m not uneducated Lovely.I have Universit...,"[mmm, uneducated, lovely, university, degree, ...","[(mmm, NN), (uneducated, VBD), (lovely, RB), (...","[mmm, qualification, sure, farage, fighting, f...",0,0,5
6,3 out of 4 reviews overturned by drs. this hap...,Rahul Singh,1136672546311401472,2019-06-06 16:33:48,['AUSvWIpic'],proudySingh,0.0,2.0,0.0,0.5859,...,0.000000,0.000770,0.030805,3 out of 4 reviews overturned by DRS This happ...,"[3, 4, reviews, overturned, drs, happens, pass...","[(3, CD), (4, CD), (reviews, NNS), (overturned...","[reviews, happens, course, university, rip, au...",0,1,5
7,how does it feel to be a student at lovely pro...,NdroPc,1136174928892444672,2019-06-05 07:36:26,[],NdroPc,0.0,0.0,0.0,0.6322,...,0.000000,0.000000,0.000000,How does it feel to be a student at Lovely Pro...,"[feel, student, lovely, professional, universi...","[(feel, VB), (student, NN), (lovely, RB), (pro...","[student, university, lpu]",0,0,5
8,answer to how does it feel to be a student at ...,RS.Pawar,1136174839608168448,2019-06-05 07:36:05,[],rahul_spawar,0.0,0.0,0.0,0.6322,...,0.000000,0.000000,0.000000,Answer to How does it feel to be a student at ...,"[answer, feel, student, lovely, professional, ...","[(answer, NN), (feel, VB), (student, NN), (lov...","[answer, student, university, lpu, rahul, pawar]",0,0,5
9,happy eid mubarak\n#eid @ lovely professional ...,HarishYemireddy,1136027341841260544,2019-06-04 21:49:59,['eid'],yharish98,0.0,0.0,0.0,0.8176,...,0.000000,0.000000,0.000000,Happy Eid Mubarak #eid Lovely Professional Uni...,"[happy, eid, mubar

In [52]:
df.to_csv('LPUWeightedSentiment.csv')